In [ ]:
using DifferentialEquations
using StochasticDelayDiffEq
using SpecialFunctions
using Plots
using XLSX
using Distributions
using SimulatedAnnealingABC
using Distances
using DataFrames
using FFTW
using CairoMakie
using PairPlots

In [ ]:
# FUNCTION FOR THE PLOTTING OF THE POSTERIORS AS A CORNER PLOT

function post_plotting(post_par, true_vals)
  pairplot(
    posterior_params,
    PairPlots.Truth(
        (;
            N_value = true_vals[1],
            T_value = true_vals[2],
            tau_value = true_vals[3],
            sigma_value = true_vals[4],
            Bmax_value = true_vals[5]
        ),
        
        label="True Values"
    )
  )
end